In [1]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

  Obtaining dependency information for qdrant_client from https://files.pythonhosted.org/packages/cf/07/b5ad4d11ed64e61aec3f7ab13a5f372b845e0d40363c0da67091c0552b40/qdrant_client-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio-tools>=1.41.0 from https://files.pythonhosted.org/packages/96/fa/099067af43926b658da01665d6a7cb98f2abd51b6504f9238fa645031d4b/grpcio_tools-1.62.1-cp310-cp310-macosx_12_0_universal2.whl.metadata
  Obtaining dependency information for h2<5,>=3 from https://files.pythonhosted.org/packages/2a/e5/db6d438da759efbb488c4f3fbdab7764492ff3c3f953132efa6b9f0e9e53/h2-4.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for hyperframe<7,>=6.0 from https://files.pythonhosted.org/packages/d7/de/85a784bcc4a3779d1753a7ec2dee5de90e18c7bcf402e71b51fcf150b129/hyperframe-6.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for hpack<5,>=4.0 from https://files.pythonhosted.org/packages/d5/34/e8b383f35b77c402d28563d2b8f83159319b5

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as skaoai
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [3]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [4]:
kernel = sk.Kernel()

In [5]:
chat_service = skaoai.AzureChatCompletion("Your Azure OpenAI Service gpt-35-turbo-16k Model Delployment Name",endpoint,api_key=api_key,api_version = "2023-12-01-preview")
embedding_gen = skaoai.AzureTextEmbedding("Your Azure OpenAI Service text-embedding-ada-002 Model Delployment Name", endpoint,api_key=api_key,api_version = "2023-12-01-preview")

In [6]:

kernel.add_service(chat_service)

kernel.add_service(embedding_gen)

In [7]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection('aboutMe')

In [8]:
# kernel.register_memory_store(memory_store=qdrant_store)
memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=embedding_gen)

In [9]:
base_vectordb = 'aboutMe'

In [10]:
await memory.save_information(base_vectordb, id="info1", text="Kinfey is Microsoft Cloud Advocate")
await memory.save_information(base_vectordb, id="info2", text="Kinfey is ex-Microsoft MVP")
await memory.save_information(base_vectordb, id="info3", text="Kinfei is AI Expert")
await memory.save_information(base_vectordb, id="info4", text="OpenAI is a company that is developing artificial general intelligence (AGI) with widely distributed economic benefits.")

In [11]:
ask = "who is kinfey？"

memories = await memory.search(
    base_vectordb, ask, limit=5, min_relevance_score=0.8
)

In [12]:
i = 0
for memory in memories:
    i = i + 1
    print(f"Top {i} Result: {memory.text} with score {memory.relevance}")

Top 1 Result: Kinfey is ex-Microsoft MVP with score 0.86921084
Top 2 Result: Kinfey is Microsoft Cloud Advocate with score 0.8400177
Top 3 Result: Kinfei is AI Expert with score 0.8256959
